# Sampling II: Euler and velocity verlet


## Content
- Euler and velocity verlet integrators
- Reweighting

## Remember jupyter notebooks
- To run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>.
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def euler(potential_gradient, size, x_init, v_init, mass=1.0, time_step=0.005):
    x, v = np.zeros(size), np.zeros(size)
    x[0], v[0] = x_init, v_init
    for i in range(1, size):
        x[i] = x[i - 1] + time_step * v[i - 1]
        v[i] = v[i - 1] - time_step * potential_gradient(x[i - 1]) / mass
    return x, v


def vv(potential_gradient, size, x_init, v_init, mass=1.0, time_step=0.005):
    x, v = np.zeros(size), np.zeros(size)
    x[0], v[0] = x_init, v_init
    nabla_u1, nabla_u0 = potential_gradient(x[0]), None
    for i in range(1, size):
        x[i] = x[i - 1] + time_step * v[i - 1] - time_step**2 * nabla_u1 / 2 / mass
        nabla_u1, nabla_u0 = potential_gradient(x[i]), nabla_u1
        v[i] = v[i - 1] - time_step * (nabla_u1 + nabla_u0) / 2 / mass
    return x, v


def harmonic_potential(x):
    return np.power(x, 2) / 2


def harmonic_potential_gradient(x):
    return x


fig, axes = plt.subplots(1, 2, figsize=(10, 5))
for ax, integrator in zip(axes.flat, (euler, vv)):
    x, v = integrator(
        harmonic_potential_gradient, 100000, 1, 0, time_step=0.001)
    ax.plot(x, v)
    ax.set_aspect('equal')
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$v$')
fig.tight_layout()

In [ ]:
def pairs(iterable):
    for x, y in zip(iterable[:-1], iterable[1:]):
        yield x, y


def chi(x, xmin, xmax):
    return np.logical_and(xmin <= x, x < xmax)


positions, _ = vv(harmonic_potential_gradient, 100000, -5, 0)
edges = np.linspace(-5, 5, 31)
centers = (edges[:-1] + edges[1:]) / 2

histogram = [np.sum(chi(positions, x, y)) / positions.size
             for x, y in pairs(edges)]

fig, ax = plt.subplots()
ax.bar(centers, [h for h in histogram], (edges[1] - edges[0]) * 0.9)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n)$')
fig.tight_layout()

In [ ]:
empirical_weights = np.zeros(len(positions))
for h, (x, y) in zip(histogram, pairs(edges)):
    empirical_weights[chi(positions, x, y)] = h
empirical_weights /= empirical_weights.sum()
    
weights = np.exp(-harmonic_potential(positions) * 1.0)
weights /= weights.sum()

histogram2 = [np.sum(weights[chi(positions, x, y)])
              for x, y in pairs(edges)]
    
corrected_weights = weights / empirical_weights
corrected_weights /= corrected_weights.sum()

histogram3 = [np.sum(corrected_weights[chi(positions, x, y)])
              for x, y in pairs(edges)]

fig, ax = plt.subplots()
ax.bar(
    centers,
    [h for h in histogram],
    (edges[1] - edges[0]) * 0.9,
    label=r'$\frac{1}{N}\sum_{n=1}^N \chi_i(x_n)$')
ax.bar(
    centers,
    [h for h in histogram2],
    (edges[1] - edges[0]) * 0.8,
    label=r'$\sum_{n=1}^N \chi_i(x_n)p(x_n)$')
ax.bar(
    centers,
    [h for h in histogram3],
    (edges[1] - edges[0]) * 0.7,
    label=r'$\sum_{n=1}^N \chi_i(x_n)p(x_n) w^{-1}(x_n)$')
ax.set_xlabel(r'$x$')
ax.legend()
fig.tight_layout()